<div class='alert alert-warning'>

SciPy's interactive examples with Jupyterlite are experimental and may not always work as expected. Execution of cells containing imports may result in large downloads (up to 60MB of content for the first import from SciPy). Load times when importing from SciPy may take roughly 10-20 seconds. If you notice any problems, feel free to open an [issue](https://github.com/scipy/scipy/issues/new/choose).

</div>

Let us consider the problem of minimizing the Rosenbrock function. This
function is implemented in `rosen` in `scipy.optimize`.


In [ ]:
import numpy as np
from scipy.optimize import rosen, differential_evolution
bounds = [(0,2), (0, 2), (0, 2), (0, 2), (0, 2)]
result = differential_evolution(rosen, bounds)
result.x, result.fun

(array([1., 1., 1., 1., 1.]), 1.9216496320061384e-19)

Now repeat, but with parallelization.


In [ ]:
result = differential_evolution(rosen, bounds, updating='deferred',
                                workers=2)
result.x, result.fun

(array([1., 1., 1., 1., 1.]), 1.9216496320061384e-19)

Let's do a constrained minimization.


In [ ]:
from scipy.optimize import LinearConstraint, Bounds

We add the constraint that the sum of ``x[0]`` and ``x[1]`` must be less
than or equal to 1.9.  This is a linear constraint, which may be written
``A @ x <= 1.9``, where ``A = array([[1, 1]])``.  This can be encoded as
a `LinearConstraint` instance:


In [ ]:
lc = LinearConstraint([[1, 1]], -np.inf, 1.9)

Specify limits using a `Bounds` object.


In [ ]:
bounds = Bounds([0., 0.], [2., 2.])
result = differential_evolution(rosen, bounds, constraints=lc,
                                seed=1)
result.x, result.fun

(array([0.96632622, 0.93367155]), 0.0011352416852625719)

Next find the minimum of the Ackley function
(https://en.wikipedia.org/wiki/Test_functions_for_optimization).


In [ ]:
def ackley(x):
    arg1 = -0.2 * np.sqrt(0.5 * (x[0] ** 2 + x[1] ** 2))
    arg2 = 0.5 * (np.cos(2. * np.pi * x[0]) + np.cos(2. * np.pi * x[1]))
    return -20. * np.exp(arg1) - np.exp(arg2) + 20. + np.e
bounds = [(-5, 5), (-5, 5)]
result = differential_evolution(ackley, bounds, seed=1)
result.x, result.fun

(array([0., 0.]), 4.440892098500626e-16)

The Ackley function is written in a vectorized manner, so the
``'vectorized'`` keyword can be employed. Note the reduced number of
function evaluations.


In [ ]:
result = differential_evolution(
    ackley, bounds, vectorized=True, updating='deferred', seed=1
)
result.x, result.fun

(array([0., 0.]), 4.440892098500626e-16)

The following custom strategy function mimics 'best1bin':


In [ ]:
def custom_strategy_fn(candidate, population, rng=None):
    parameter_count = population.shape(-1)
    mutation, recombination = 0.7, 0.9
    trial = np.copy(population[candidate])
    fill_point = rng.choice(parameter_count)

    pool = np.arange(len(population))
    rng.shuffle(pool)

    # two unique random numbers that aren't the same, and
    # aren't equal to candidate.
    idxs = []
    while len(idxs) < 2 and len(pool) > 0:
        idx = pool[0]
        pool = pool[1:]
        if idx != candidate:
            idxs.append(idx)

    r0, r1 = idxs[:2]

    bprime = (population[0] + mutation *
              (population[r0] - population[r1]))

    crossovers = rng.uniform(size=parameter_count)
    crossovers = crossovers < recombination
    crossovers[fill_point] = True
    trial = np.where(crossovers, bprime, trial)
    return trial